## Language Detection Model

### Importing Packages

In [1]:
import re
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import warnings
warnings.simplefilter("ignore")

### Model Building

Simple and quick way to develop the model because our goal is to understand, how to deploy the model on AWS ECS.

In [2]:
# Loading the dataset
data = pd.read_csv("Language Detection.csv")

In [3]:
data.head()

,Text,Language
0,"Nature, in the broadest sense, is the natural...",English
1,"""Nature"" can refer to the phenomena of the phy...",English
2,"The study of nature is a large, if not the onl...",English
3,"Although humans are part of nature, human acti...",English
4,[1] The word nature is borrowed from the Old F...,English


In [4]:
X = data["Text"]
y = data["Language"]

In [5]:
le = LabelEncoder()
y = le.fit_transform(y)

In [6]:
le.classes_

array(['Arabic', 'Danish', 'Dutch', 'English', 'French', 'German',
       'Greek', 'Hindi', 'Italian', 'Kannada', 'Malayalam', 'Portugeese',
       'Russian', 'Spanish', 'Sweedish', 'Tamil', 'Turkish'], dtype=object)

In [7]:
data_list = []

for text in X:
    text = re.sub(r'[!@#$(),\n"%^*?\:;~`0-9]', ' ', text)
    text = re.sub(r'[[]]', ' ', text)
    text = text.lower()
    data_list.append(text)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [9]:
# creating bag of words using countvectorizer

cv = CountVectorizer()
cv.fit(X_train)

x_train = cv.transform(X_train).toarray()
x_test  = cv.transform(X_test).toarray()

In [10]:
model = MultinomialNB()
model.fit(x_train, y_train)

MultinomialNB()

In [11]:
y_pred = model.predict(x_test)

In [12]:
ac = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
cr = classification_report(y_test, y_pred)

In [13]:
print("Accuracy =", round(ac, 3)*100, '%')

Accuracy = 97.8 %


In [14]:
pipe = Pipeline([('vectorizer', cv), ('multinomialNB', model)])
pipe.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()),
                ('multinomialNB', MultinomialNB())])

In [15]:
y_pred2 = pipe.predict(X_test)
ac2 = accuracy_score(y_test, y_pred2)

In [16]:
print("Accuracy =", round(ac2, 3)*100, '%')

Accuracy = 97.8 %


### Saving the ML Model (Serialization) - Imp 

In [17]:
with open('lang_trained_pipeline.pkl','wb') as f:
    pickle.dump(pipe, f)

### Model Inference using Serialized file

In [18]:
model_pk = pickle.load(open('lang_trained_pipeline.pkl','rb'))

In [19]:
text = "Ciao, come stai?"

detect = model_pk.predict([text])

In [20]:
detect

array([8])

In [21]:
le.classes_[detect[0]], detect

('Italian', array([8]))